# Análise de Performance por Categoria de Produto - Olist E-Commerce

**Autor:** André Bomfim  
**Data:** Novembro 2024  
**Objetivo:** Analisar receita, ticket médio, margem e sazonalidade por categoria para identificar produtos estratégicos e oportunidades de crescimento

---

## Índice

1. [Setup e Configuração](#1-setup)
2. [Extração e Performance Geral](#2-overview)
3. [Análise de Preço e Margem](#3-pricing)
4. [Evolução Temporal e Sazonalidade](#4-temporal)
5. [Análise de Qualidade (NPS)](#5-nps)
6. [Análise Geográfica](#6-geographic)
7. [Análise de Clientes](#7-customers)
8. [Insights e Recomendações](#8-insights)

## 1. Setup e Configuração

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv
import os

warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuração do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Carregar variáveis de ambiente
load_dotenv()

# Configuração BigQuery
PROJECT_ID = os.getenv('GCP_PROJECT_ID')
DATASET_ID = os.getenv('GCP_DATASET_ID', 'olist_ecommerce')

# Cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Criar diretório para imagens
os.makedirs('../docs/images', exist_ok=True)

print(f"✓ Setup completo - Projeto: {PROJECT_ID}, Dataset: {DATASET_ID}")

In [ ]:
def query_bigquery(query: str) -> pd.DataFrame:
    """Helper para executar queries no BigQuery"""
    return client.query(query).to_dataframe()

def save_plot(fig, filename):
    """Salvar gráfico com tratamento de erro"""
    try:
        fig.savefig(f'../docs/images/{filename}', dpi=300, bbox_inches='tight')
        print(f"Gráfico salvo: docs/images/{filename}")
    except Exception as e:
        print(f" Erro ao salvar gráfico: {e}")

def classify_category(row, top_20_cutoff, top_50_cutoff):
    """
    Classifica categoria baseado em receita acumulada
    Champions: top 20% (80% receita)
    Opportunity: middle 30%
    Long Tail: bottom 50%
    """
    if row['cumulative_revenue_pct'] <= top_20_cutoff:
        return 'Champions'
    elif row['cumulative_revenue_pct'] <= top_50_cutoff:
        return 'Opportunity'
    else:
        return 'Long Tail'

def calculate_growth_rate(current, previous):
    """Calcula taxa de crescimento percentual"""
    if previous == 0 or pd.isna(previous):
        return np.nan
    return ((current - previous) / previous) * 100

def calculate_priority_score(revenue_pct, growth_rate, nps, revenue_rank):
    """
    Score de prioridade para investimento
    Maior score = maior prioridade
    """
    # Normalizar métricas (0-1)
    revenue_norm = revenue_pct / 100
    growth_norm = max(0, min(growth_rate / 100, 1)) if not pd.isna(growth_rate) else 0
    nps_norm = nps / 5.0 if not pd.isna(nps) else 0
    rank_norm = 1 - (revenue_rank / 100)  # Inverter rank
    
    # Ponderação
    score = (revenue_norm * 0.4 + 
             growth_norm * 0.3 + 
             nps_norm * 0.2 + 
             rank_norm * 0.1)
    
    return score * 100

def categorize_price_volume(row, median_price, median_volume):
    """Matriz 2x2: High/Low Price x High/Low Volume"""
    price_high = row['avg_price'] > median_price
    volume_high = row['total_orders'] > median_volume
    
    if price_high and volume_high:
        return 'Premium_HighVolume'
    elif price_high and not volume_high:
        return 'Premium_LowVolume'
    elif not price_high and volume_high:
        return 'Popular_HighVolume'
    else:
        return 'LowValue_LowVolume'

print("Funções auxiliares carregadas")

## 2. Extração e Performance Geral

In [ ]:
# Query Performance por Categoria
query_category_base = f"""
WITH category_base AS (
    SELECT 
        COALESCE(p.product_category_name, 'unknown') as product_category_name,
        COUNT(DISTINCT o.order_id) as total_orders,
        SUM(oi.price) as total_revenue,
        AVG(oi.price) as avg_price,
        AVG(oi.freight_value) as avg_freight,
        COUNT(oi.order_id) as total_items_sold,
        COUNT(DISTINCT c.customer_unique_id) as unique_customers,
        COUNT(DISTINCT s.seller_id) as unique_sellers,
        AVG(CAST(r.review_score AS FLOAT64)) as avg_review_score,
        COUNT(r.review_id) as review_count
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.order_items` oi ON o.order_id = oi.order_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.products` p ON oi.product_id = p.product_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.sellers` s ON oi.seller_id = s.seller_id
    LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.order_reviews` r ON o.order_id = r.order_id
    WHERE o.order_status = 'delivered'
    AND oi.price > 0
    GROUP BY p.product_category_name
    HAVING COUNT(DISTINCT o.order_id) >= 50
)
SELECT
    *,
    total_revenue / total_orders as avg_ticket,
    total_revenue / unique_customers as revenue_per_customer
FROM category_base
ORDER BY total_revenue DESC
"""

df_category = query_bigquery(query_category_base)

# Calcular métricas adicionais
total_revenue = df_category['total_revenue'].sum()
df_category['revenue_pct'] = (df_category['total_revenue'] / total_revenue) * 100
df_category['cumulative_revenue_pct'] = df_category['revenue_pct'].cumsum()
df_category['margin_estimate'] = df_category['avg_price'] - df_category['avg_freight']

# Classificar categorias (Pareto)
top_20_cutoff = 80  # 80% da receita
top_50_cutoff = 95  # 95% da receita (middle 30%)
df_category['segment'] = df_category.apply(
    lambda x: classify_category(x, top_20_cutoff, top_50_cutoff), axis=1
)

# Adicionar rank
df_category['revenue_rank'] = df_category['total_revenue'].rank(ascending=False)

# TRATAMENTO DE CATEGORIAS VAZIAS
if 'unknown' in df_category['product_category_name'].values:
    unknown_count = df_category[df_category['product_category_name'] == 'unknown'].shape[0]
    unknown_revenue = df_category[df_category['product_category_name'] == 'unknown']['total_revenue'].sum()
    print(f" {unknown_count} produtos sem categoria (R$ {unknown_revenue:,.2f} - {unknown_revenue/total_revenue*100:.1f}% da receita)")

print(f" {len(df_category):,} categorias carregadas (≥50 pedidos)")
print(f" Receita total: R$ {total_revenue:,.2f}")
print(f" Categorias Champions: {(df_category['segment'] == 'Champions').sum()}")
print(f" Categorias Opportunity: {(df_category['segment'] == 'Opportunity').sum()}")
print(f" Categorias Long Tail: {(df_category['segment'] == 'Long Tail').sum()}")

In [ ]:
# Visualização Performance Geral
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Gráfico 1: Top 20 categorias por receita
top_20 = df_category.head(20).sort_values('total_revenue', ascending=True)
bars1 = axes[0, 0].barh(range(len(top_20)), top_20['total_revenue']/1000, 
                       color='skyblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_yticks(range(len(top_20)))
axes[0, 0].set_yticklabels(top_20['product_category_name'])
axes[0, 0].set_xlabel('Receita (R$ mil)')
axes[0, 0].set_title('Top 20 Categorias por Receita')

# Gráfico 2: Curva de Pareto
pareto_data = df_category.copy()
pareto_data['cumulative_categories_pct'] = (pareto_data.index + 1) / len(pareto_data) * 100

axes[0, 1].plot(pareto_data['cumulative_categories_pct'], pareto_data['cumulative_revenue_pct'], 
               linewidth=3, label='Curva de Pareto', color='blue')
axes[0, 1].plot([0, 100], [0, 100], '--', color='gray', alpha=0.7, label='Igualdade')
axes[0, 1].axhline(80, color='red', linestyle='--', alpha=0.7, label='80% Receita')
axes[0, 1].axvline(20, color='red', linestyle='--', alpha=0.7, label='20% Categorias')
axes[0, 1].set_xlabel('% Acumulado de Categorias')
axes[0, 1].set_ylabel('% Acumulado de Receita')
axes[0, 1].set_title('Curva de Pareto: Categorias vs Receita')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Gráfico 3: Scatter preço médio vs volume (bolhas = receita)
scatter = axes[1, 0].scatter(df_category['avg_price'], df_category['total_orders'], 
                            s=df_category['total_revenue']/1000, 
                            c=df_category['avg_review_score'], 
                            cmap='RdYlGn', alpha=0.6)
axes[1, 0].set_xlabel('Preço Médio (R$)')
axes[1, 0].set_ylabel('Volume de Pedidos')
axes[1, 0].set_title('Preço Médio vs Volume (tamanho=receita, cor=NPS)')
plt.colorbar(scatter, ax=axes[1, 0], label='NPS Médio')

# Gráfico 4: Pie chart Top 5 categorias + Others
top_5 = df_category.head(5)
others_revenue = df_category[~df_category.index.isin(top_5.index)]['total_revenue'].sum()
pie_data = pd.concat([top_5['total_revenue'], pd.Series([others_revenue], index=['Others'])])
pie_labels = list(top_5['product_category_name']) + ['Others']

axes[1, 1].pie(pie_data, labels=pie_labels, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Distribuição de Receita: Top 5 Categorias + Others')

plt.tight_layout()
save_plot(fig, 'category_performance_overview.png')
plt.show()

## 3. Análise de Preço e Margem 

In [ ]:
# Query Detalhada de Preços
top_10_categories_list = df_category.head(10)['product_category_name'].tolist()
categories_formatted = ','.join([f"'{cat}'" for cat in top_10_categories_list])

query_pricing_detail = f"""
SELECT 
    p.product_category_name,
    oi.price,
    oi.freight_value
FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
JOIN `{PROJECT_ID}.{DATASET_ID}.order_items` oi ON o.order_id = oi.order_id
JOIN `{PROJECT_ID}.{DATASET_ID}.products` p ON oi.product_id = p.product_id
WHERE o.order_status = 'delivered'
AND oi.price > 0
AND p.product_category_name IN ({categories_formatted})
"""

df_pricing_detail = query_bigquery(query_pricing_detail)
print(f" {len(df_pricing_detail):,} registros de preços carregados")

# Calcular medianas para categorização
median_price = df_category['avg_price'].median()
median_volume = df_category['total_orders'].median()

# Aplicar categorização preço x volume
df_category['price_volume_segment'] = df_category.apply(
    lambda x: categorize_price_volume(x, median_price, median_volume), axis=1
)

print(f"\n SEGMENTAÇÃO PREÇO x VOLUME:")
segment_counts = df_category['price_volume_segment'].value_counts()
for segment, count in segment_counts.items():
    print(f"  • {segment}: {count} categorias")

In [ ]:
# Visualização Análise de Preço e Margem
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Gráfico 1: Boxplot distribuição preços top 10 categorias
top_10_categories = df_category.head(10)['product_category_name'].tolist()
df_top10_prices = df_pricing_detail[df_pricing_detail['product_category_name'].isin(top_10_categories)]

sns.boxplot(data=df_top10_prices, x='product_category_name', y='price', ax=axes[0, 0])
axes[0, 0].set_xlabel('Categoria')
axes[0, 0].set_ylabel('Preço (R$)')
axes[0, 0].set_title('Distribuição de Preços - Top 10 Categorias')
axes[0, 0].tick_params(axis='x', rotation=45)

# Gráfico 2: Scatter preço vs ticket médio
scatter2 = axes[0, 1].scatter(df_category['avg_price'], df_category['avg_ticket'], 
                             s=df_category['total_orders']/10, 
                             c=df_category['segment'].map({'Champions': 'green', 'Opportunity': 'orange', 'Long Tail': 'red'}), 
                             alpha=0.6)
axes[0, 1].set_xlabel('Preço Médio Produto (R$)')
axes[0, 1].set_ylabel('Ticket Médio Pedido (R$)')
axes[0, 1].set_title('Preço Médio vs Ticket Médio (tamanho=volume, cor=segmento)')

# Gráfico 3: Bar chart frete médio por categoria (top 10)
top_10_freight = df_category.head(10).sort_values('avg_freight', ascending=True)
bars3 = axes[1, 0].barh(range(len(top_10_freight)), top_10_freight['avg_freight'], 
                       color='orange', alpha=0.7, edgecolor='black')
axes[1, 0].set_yticks(range(len(top_10_freight)))
axes[1, 0].set_yticklabels(top_10_freight['product_category_name'])
axes[1, 0].set_xlabel('Frete Médio (R$)')
axes[1, 0].set_title('Top 10 Categorias - Frete Médio')

# Gráfico 4: Heatmap preço vs volume (matriz 2x2)
price_volume_pivot = pd.crosstab(df_category['price_volume_segment'], 'count')
segment_order = ['Premium_HighVolume', 'Premium_LowVolume', 'Popular_HighVolume', 'LowValue_LowVolume']
price_volume_pivot = price_volume_pivot.reindex(segment_order)

im = axes[1, 1].imshow([price_volume_pivot['count'].values], cmap='YlOrRd', aspect='auto')
axes[1, 1].set_yticks([0])
axes[1, 1].set_yticklabels([''])
axes[1, 1].set_xticks(range(len(segment_order)))
axes[1, 1].set_xticklabels(segment_order, rotation=45)
axes[1, 1].set_title('Matriz Preço x Volume: Distribuição de Categorias')

# Anotar valores
for i, count in enumerate(price_volume_pivot['count'].values):
    axes[1, 1].text(i, 0, f'{count} cat', ha='center', va='center', 
                   color='white' if count > price_volume_pivot['count'].median() else 'black',
                   fontweight='bold')

plt.tight_layout()
save_plot(fig, 'category_pricing_analysis.png')
plt.show()

## 4. Evolução Temporal e Sazonalidade 

In [ ]:
# Query Evolução Temporal
top_5_categories_list = df_category.head(5)['product_category_name'].tolist()
categories_formatted_temporal = ','.join([f"'{cat}'" for cat in top_5_categories_list])

query_category_monthly = f"""
WITH monthly_data AS (
    SELECT 
        FORMAT_DATE('%Y-%m', o.order_purchase_timestamp) as year_month,
        p.product_category_name,
        SUM(oi.price) as monthly_revenue,
        COUNT(DISTINCT o.order_id) as monthly_orders
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.order_items` oi ON o.order_id = oi.order_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.products` p ON oi.product_id = p.product_id
    WHERE o.order_status = 'delivered'
    AND oi.price > 0
    AND p.product_category_name IN ({categories_formatted_temporal})
    GROUP BY year_month, p.product_category_name
)
SELECT
    year_month,
    product_category_name,
    monthly_revenue,
    monthly_orders
FROM monthly_data
ORDER BY year_month, product_category_name
"""

df_category_monthly = query_bigquery(query_category_monthly)

# VALIDAÇÃO DADOS TEMPORAIS
if df_category_monthly.empty:
    print(" Sem dados temporais disponíveis para análise temporal")
    # Criar dataframe vazio para evitar erros
    df_category_monthly = pd.DataFrame(columns=['year_month', 'product_category_name', 'monthly_revenue', 'monthly_orders'])
else:
    print(f" {len(df_category_monthly):,} registros mensais carregados")
    print(f" Período analisado: {df_category_monthly['year_month'].min()} a {df_category_monthly['year_month'].max()}")
    print(f" Meses únicos: {df_category_monthly['year_month'].nunique()}")

# Calcular growth rate
df_category_monthly = df_category_monthly.sort_values(['product_category_name', 'year_month'])
df_category_monthly['growth_rate_mom'] = df_category_monthly.groupby('product_category_name')['monthly_revenue'].pct_change() * 100

# Pivot para análise
monthly_pivot = df_category_monthly.pivot_table(
    index='year_month',
    columns='product_category_name',
    values='monthly_revenue',
    aggfunc='sum'
).fillna(0)

print("\n EVOLUÇÃO MENSAL - TOP 5 CATEGORIAS:")
print(monthly_pivot.tail())

In [ ]:
# Visualização Evolução Temporal
plt.close('all')
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Gráfico 1: Line plot evolução mensal top 5 categorias
for category in monthly_pivot.columns:
    axes[0].plot(monthly_pivot.index, monthly_pivot[category], 
                marker='o', linewidth=2, label=category, markersize=4)

axes[0].set_xlabel('Mês')
axes[0].set_ylabel('Receita Mensal (R$)')
axes[0].set_title('Evolução Mensal - Top 5 Categorias por Receita')
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Destacar Black Friday (Novembro)
for i, month in enumerate(monthly_pivot.index):
    if '-11' in month:  # Novembro
        axes[0].axvline(i, color='red', linestyle='--', alpha=0.5, label='Black Friday' if i == 0 else "")

# Gráfico 2: Bar chart growth rate (último mês disponível)
latest_month = df_category_monthly['year_month'].max()
latest_growth = df_category_monthly[df_category_monthly['year_month'] == latest_month]

if not latest_growth.empty:
    latest_growth = latest_growth.sort_values('growth_rate_mom', ascending=True)
    bars = axes[1].bar(range(len(latest_growth)), latest_growth['growth_rate_mom'], 
                     color=['green' if x > 0 else 'red' for x in latest_growth['growth_rate_mom']],
                     alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Categoria')
    axes[1].set_ylabel('Growth Rate MoM (%)')
    axes[1].set_title(f'Taxa de Crescimento Mensal - {latest_month}')
    axes[1].set_xticks(range(len(latest_growth)))
    axes[1].set_xticklabels(latest_growth['product_category_name'], rotation=45)
    
    # Adicionar valores nas barras
    for bar, value in zip(bars, latest_growth['growth_rate_mom']):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + (1 if value > 0 else -3), 
                   f'{value:.1f}%', ha='center', va='bottom' if value > 0 else 'top', 
                   fontweight='bold', color='green' if value > 0 else 'red')

plt.tight_layout()
save_plot(fig, 'category_temporal_analysis.png')
plt.show()

## 5. Análise de Qualidade (NPS) 

In [ ]:
# Preparar dados NPS
df_nps_analysis = df_category[df_category['review_count'] >= 10].copy()  # Mínimo 10 reviews

# Calcular priority score
df_nps_analysis['priority_score'] = df_nps_analysis.apply(
    lambda x: calculate_priority_score(
        x['revenue_pct'], 
        10,  # Placeholder para growth rate - usaríamos dados reais se disponíveis
        x['avg_review_score'], 
        x['revenue_rank']
    ), axis=1
)

# Top e Bottom por NPS
top_nps = df_nps_analysis.nlargest(10, 'avg_review_score')
bottom_nps = df_nps_analysis.nsmallest(10, 'avg_review_score')

print(" TOP 10 CATEGORIAS POR NPS:")
print(top_nps[['product_category_name', 'avg_review_score', 'review_count', 'total_revenue']])
print(f"\n  BOTTOM 10 CATEGORIAS POR NPS:")
print(bottom_nps[['product_category_name', 'avg_review_score', 'review_count', 'total_revenue']])

In [ ]:
# Visualização Análise NPS
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Gráfico 1: Top 10 categorias por NPS
top_nps_sorted = top_nps.sort_values('avg_review_score', ascending=True)
bars1 = axes[0, 0].barh(range(len(top_nps_sorted)), top_nps_sorted['avg_review_score'], 
                       color='green', alpha=0.7, edgecolor='black')
axes[0, 0].set_yticks(range(len(top_nps_sorted)))
axes[0, 0].set_yticklabels(top_nps_sorted['product_category_name'])
axes[0, 0].set_xlabel('NPS Médio')
axes[0, 0].set_title('Top 10 Categorias - Melhor NPS')
axes[0, 0].set_xlim(0, 5)

# Gráfico 2: Bottom 10 categorias por NPS
bottom_nps_sorted = bottom_nps.sort_values('avg_review_score', ascending=False)
bars2 = axes[0, 1].barh(range(len(bottom_nps_sorted)), bottom_nps_sorted['avg_review_score'], 
                       color='red', alpha=0.7, edgecolor='black')
axes[0, 1].set_yticks(range(len(bottom_nps_sorted)))
axes[0, 1].set_yticklabels(bottom_nps_sorted['product_category_name'])
axes[0, 1].set_xlabel('NPS Médio')
axes[0, 1].set_title('Bottom 10 Categorias - Pior NPS')
axes[0, 1].set_xlim(0, 5)

# Gráfico 3: Scatter NPS vs receita (bolhas = volume reviews)
scatter3 = axes[1, 0].scatter(df_nps_analysis['avg_review_score'], df_nps_analysis['total_revenue'], 
                             s=df_nps_analysis['review_count']/10, 
                             c=df_nps_analysis['priority_score'], 
                             cmap='RdYlGn', alpha=0.6)
axes[1, 0].set_xlabel('NPS Médio')
axes[1, 0].set_ylabel('Receita Total (R$)')
axes[1, 0].set_title('NPS vs Receita (tamanho=volume reviews, cor=priority score)')
plt.colorbar(scatter3, ax=axes[1, 0], label='Priority Score')

# Gráfico 4: Violin plot distribuição NPS top 5 categorias
top_5_categories = df_category.head(5)['product_category_name'].tolist()

# Query para dados detalhados de reviews
top_5_categories_formatted = ','.join([f"'{cat}'" for cat in top_5_categories])
query_review_detail = f"""
SELECT 
    p.product_category_name,
    r.review_score
FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
JOIN `{PROJECT_ID}.{DATASET_ID}.order_items` oi ON o.order_id = oi.order_id
JOIN `{PROJECT_ID}.{DATASET_ID}.products` p ON oi.product_id = p.product_id
JOIN `{PROJECT_ID}.{DATASET_ID}.order_reviews` r ON o.order_id = r.order_id
WHERE o.order_status = 'delivered'
AND p.product_category_name IN ({top_5_categories_formatted})
AND r.review_score IS NOT NULL
"""

df_review_detail = query_bigquery(query_review_detail)

if not df_review_detail.empty:
    sns.violinplot(data=df_review_detail, x='product_category_name', y='review_score', 
                  ax=axes[1, 1], palette='Set2')
    axes[1, 1].set_xlabel('Categoria')
    axes[1, 1].set_ylabel('Review Score')
    axes[1, 1].set_title('Distribuição de NPS - Top 5 Categorias')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
save_plot(fig, 'category_nps_analysis.png')
plt.show()

## 6. Análise Geográfica

In [ ]:
# Query Performance Geográfica
top_10_categories_geo = df_category.head(10)['product_category_name'].tolist()
top_10_categories_formatted = ','.join([f"'{cat}'" for cat in top_10_categories_geo])

# Estados brasileiros mais populosos
brazilian_states = ['SP', 'RJ', 'MG', 'RS', 'PR', 'BA', 'SC', 'GO']
states_formatted = ','.join([f"'{state}'" for state in brazilian_states])

query_category_geographic = f"""
WITH geographic_data AS (
    SELECT 
        p.product_category_name,
        c.customer_state,
        SUM(oi.price) as revenue,
        COUNT(DISTINCT o.order_id) as orders
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.order_items` oi ON o.order_id = oi.order_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.products` p ON oi.product_id = p.product_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
    WHERE o.order_status = 'delivered'
    AND oi.price > 0
    AND p.product_category_name IN ({top_10_categories_formatted})
    AND c.customer_state IN ({states_formatted})
    GROUP BY p.product_category_name, c.customer_state
    HAVING COUNT(DISTINCT o.order_id) >= 10
)
SELECT
    product_category_name,
    customer_state,
    revenue,
    orders,
    revenue / SUM(revenue) OVER(PARTITION BY customer_state) * 100 as market_share_pct
FROM geographic_data
ORDER BY customer_state, revenue DESC
"""

df_category_geographic = query_bigquery(query_category_geographic)
print(f"✓ {len(df_category_geographic):,} registros geográficos carregados")

# Preparar dados para heatmap
geo_pivot = df_category_geographic.pivot_table(
    index='product_category_name',
    columns='customer_state',
    values='market_share_pct',
    aggfunc='mean'
).fillna(0)

print("\n HEATMAP GEOGRÁFICO - PRINCIPAIS DADOS:")
print(geo_pivot.head())

In [ ]:
# Visualização Análise Geográfica
plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Gráfico 1: Heatmap categorias x estados
if not geo_pivot.empty:
    im = axes[0].imshow(geo_pivot, cmap='YlOrRd', aspect='auto')
    axes[0].set_xlabel('Estado')
    axes[0].set_ylabel('Categoria')
    axes[0].set_title('Heatmap: Market Share por Categoria e Estado (%)')
    axes[0].set_xticks(range(len(geo_pivot.columns)))
    axes[0].set_xticklabels(geo_pivot.columns, rotation=45)
    axes[0].set_yticks(range(len(geo_pivot.index)))
    axes[0].set_yticklabels(geo_pivot.index)
    plt.colorbar(im, ax=axes[0], label='Market Share (%)')
    
    # Anotar valores
    for i in range(len(geo_pivot.index)):
        for j in range(len(geo_pivot.columns)):
            value = geo_pivot.iloc[i, j]
            if value > 5:  # Apenas valores significativos
                axes[0].text(j, i, f'{value:.0f}%', 
                           ha='center', va='center', 
                           color='white' if value > 15 else 'black',
                           fontsize=8)

# Gráfico 2: Stacked bar market share categorias por região
# Agrupar estados por região (simplificado)
region_map = {
    'SP': 'Sudeste', 'RJ': 'Sudeste', 'MG': 'Sudeste', 'ES': 'Sudeste',
    'RS': 'Sul', 'SC': 'Sul', 'PR': 'Sul',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'BA': 'Nordeste', 'PE': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste',
    'PA': 'Norte', 'AM': 'Norte', 'RO': 'Norte', 'AC': 'Norte'
}

df_category_geographic['region'] = df_category_geographic['customer_state'].map(region_map)
df_region = df_category_geographic.groupby(['region', 'product_category_name'])['revenue'].sum().reset_index()

# Calcular market share por região
region_totals = df_region.groupby('region')['revenue'].transform('sum')
df_region['market_share'] = (df_region['revenue'] / region_totals) * 100

# Pivot para stacked bar
region_pivot = df_region.pivot_table(
    index='region',
    columns='product_category_name',
    values='market_share',
    aggfunc='sum'
).fillna(0)

# Manter apenas top 5 categorias por região
top_categories_per_region = region_pivot.sum().nlargest(5).index
region_pivot_top = region_pivot[top_categories_per_region]

region_pivot_top.plot(kind='bar', stacked=True, ax=axes[1], 
                     color=['blue', 'green', 'red', 'orange', 'purple'])
axes[1].set_xlabel('Região')
axes[1].set_ylabel('Market Share (%)')
axes[1].set_title('Market Share por Categoria e Região')
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
save_plot(fig, 'category_geographic_analysis.png')
plt.show()

## 7. Análise de Clientes 

In [ ]:
# Query Análise de Clientes
top_15_categories_customer = df_category.head(15)['product_category_name'].tolist()
top_15_categories_formatted = ','.join([f"'{cat}'" for cat in top_15_categories_customer])

query_customer_analysis = f"""
WITH customer_categories AS (
    SELECT 
        c.customer_unique_id,
        p.product_category_name,
        COUNT(DISTINCT o.order_id) as order_count,
        SUM(oi.price) as total_spent
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.order_items` oi ON o.order_id = oi.order_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.products` p ON oi.product_id = p.product_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
    WHERE o.order_status = 'delivered'
    AND oi.price > 0
    AND p.product_category_name IN ({top_15_categories_formatted})
    GROUP BY c.customer_unique_id, p.product_category_name
)
SELECT
    product_category_name,
    COUNT(DISTINCT customer_unique_id) as unique_customers,
    AVG(total_spent) as avg_revenue_per_customer,
    SUM(CASE WHEN order_count >= 2 THEN 1 ELSE 0 END) as repeat_customers,
    COUNT(DISTINCT customer_unique_id) as total_customers
FROM customer_categories
GROUP BY product_category_name
HAVING COUNT(DISTINCT customer_unique_id) >= 10
"""

df_customer_analysis = query_bigquery(query_customer_analysis)

# Calcular métricas de fidelização
df_customer_analysis['repeat_purchase_rate'] = (df_customer_analysis['repeat_customers'] / df_customer_analysis['total_customers']) * 100

# Combinar com dados principais
df_customer_full = pd.merge(df_category, df_customer_analysis, on='product_category_name', how='left')

print("👥 ANÁLISE DE CLIENTES POR CATEGORIA:")
print(f"  • Categorias com dados de clientes: {len(df_customer_analysis)}")
print(f"  • Taxa de recompra média: {df_customer_analysis['repeat_purchase_rate'].mean():.1f}%")
print(f"  • LTV médio por categoria: R$ {df_customer_analysis['avg_revenue_per_customer'].mean():.2f}")

# Top categorias por LTV
top_ltv = df_customer_full.nlargest(10, 'avg_revenue_per_customer')
print(f"\n TOP 5 CATEGORIAS POR LTV:")
print(top_ltv[['product_category_name', 'avg_revenue_per_customer', 'repeat_purchase_rate']].head())

## 8. Insights e Recomendações {#8-insights}

In [ ]:
# Sumário Executivo Final
print("\n" + "="*80)
print(" RESUMO EXECUTIVO - ANÁLISE DE PERFORMANCE POR CATEGORIA")
print("="*80)

# Coletar métricas finais
total_categories = len(df_category)
champions_categories = df_category[df_category['segment'] == 'Champions']
opportunity_categories = df_category[df_category['segment'] == 'Opportunity']
long_tail_categories = df_category[df_category['segment'] == 'Long Tail']

# Top categorias
top_category = df_category.iloc[0]
best_nps_category = df_nps_analysis.nlargest(1, 'avg_review_score').iloc[0]
highest_growth_category = df_category.nlargest(1, 'total_orders').iloc[0]  # Proxy para crescimento

print(f"\n MÉTRICAS GLOBAIS:")
print(f"   • Total categorias analisadas: {total_categories}")
print(f"   • Receita total: R$ {total_revenue:,.2f}")
print(f"   • Ticket médio geral: R$ {df_category['avg_ticket'].mean():.2f}")
print(f"   • NPS médio geral: {df_category['avg_review_score'].mean():.2f}")

print(f"\n ANÁLISE DE PARETO:")
print(f"   • Categorias Champions: {len(champions_categories)} ({len(champions_categories)/total_categories*100:.1f}%) - {champions_categories['revenue_pct'].sum():.1f}% receita")
print(f"   • Categorias Opportunity: {len(opportunity_categories)} ({len(opportunity_categories)/total_categories*100:.1f}%) - {opportunity_categories['revenue_pct'].sum():.1f}% receita")
print(f"   • Long Tail: {len(long_tail_categories)} ({len(long_tail_categories)/total_categories*100:.1f}%) - {long_tail_categories['revenue_pct'].sum():.1f}% receita")

print(f"\n CATEGORIAS DESTAQUE:")
print(f"   • Maior receita: {top_category['product_category_name']} (R$ {top_category['total_revenue']:,.2f} - {top_category['revenue_pct']:.1f}%)")
print(f"   • Melhor NPS: {best_nps_category['product_category_name']} ({best_nps_category['avg_review_score']:.2f})")
print(f"   • Maior volume: {highest_growth_category['product_category_name']} ({highest_growth_category['total_orders']:,} pedidos)")

print(f"\n ANÁLISE DE PREÇOS:")
print(f"   • Categoria mais cara: {df_category.nlargest(1, 'avg_price').iloc[0]['product_category_name']} (R$ {df_category['avg_price'].max():.2f})")
print(f"   • Melhor margem estimada: {df_category.nlargest(1, 'margin_estimate').iloc[0]['product_category_name']} (R$ {df_category['margin_estimate'].max():.2f})")
print(f"   • Segmento Premium_HighVolume: {(df_category['price_volume_segment'] == 'Premium_HighVolume').sum()} categorias")

print(f"\n ANÁLISE DE CLIENTES:")
if not df_customer_analysis.empty:
    print(f"   • Maior LTV: {df_customer_full.nlargest(1, 'avg_revenue_per_customer').iloc[0]['product_category_name']} (R$ {df_customer_full['avg_revenue_per_customer'].max():.2f})")
    print(f"   • Maior taxa recompra: {df_customer_full.nlargest(1, 'repeat_purchase_rate').iloc[0]['product_category_name']} ({df_customer_full['repeat_purchase_rate'].max():.1f}%)")

print(f"\n" + "="*80)
print(" RECOMENDAÇÕES ESTRATÉGICAS")
print("="*80)

print(f"\n PRIORIDADE 1: INVESTIR EM CATEGORIAS CHAMPIONS")
champions_list = champions_categories['product_category_name'].tolist()
print(f"   • Categorias: {', '.join(champions_list[:3])}...")
print(f"   • Ação: Marketing agressivo, expansão de estoque, featured placement")
print(f"   • Meta: Manter >80% participação receita")

print(f"\n PRIORIDADE 2: EXPANDIR CATEGORIAS HIGH-POTENTIAL")
high_potential = df_nps_analysis[
    (df_nps_analysis['avg_review_score'] > 4.0) & 
    (df_nps_analysis['revenue_pct'] < 2.0)
].nlargest(3, 'priority_score')

if not high_potential.empty:
    print(f"   • Categorias: {', '.join(high_potential['product_category_name'].tolist())}")
    print(f"   • Ação: Aumentar mix produtos, campanhas específicas, parcerias")
    print(f"   • Meta: Dobrar receita em 6 meses")

print(f"\n PRIORIDADE 3: OTIMIZAR CATEGORIAS PROBLEMÁTICAS")
problematic = df_nps_analysis[
    (df_nps_analysis['avg_review_score'] < 3.5) & 
    (df_nps_analysis['total_orders'] > 100)
]

if not problematic.empty:
    print(f"   • Categorias críticas: {problematic.shape[0]} com NPS baixo e volume alto")
    print(f"   • Ação: Revisão qualidade, feedback análise, treinamento vendedores")
    print(f"   • Meta: Aumentar NPS para >4.0 em 3 meses")

print(f"\n PRIORIDADE 4: OTIMIZAÇÃO DE PREÇOS")
elastic_categories = df_category[
    (df_category['price_volume_segment'].isin(['Premium_LowVolume', 'LowValue_LowVolume'])) &
    (df_category['total_orders'] > 50)
]
print(f"   • Categorias para revisão de preço: {elastic_categories.shape[0]}")
print(f"   • Ação: Testes A/B pricing, bundles, promoções estratégicas")
print(f"   • Meta: Aumentar conversão em 15%")

print(f"\n PRIORIDADE 5: EXPANSÃO GEOGRÁFICA")
regional_opportunities = df_category_geographic[
    df_category_geographic['market_share_pct'] < 5
].groupby('product_category_name')['revenue'].sum().nlargest(3)

if not regional_opportunities.empty:
    print(f"   • Categorias sub-exploradas: {', '.join(regional_opportunities.index.tolist())}")
    print(f"   • Ação: Campanhas regionais, parcerias locais, logística otimizada")
    print(f"   • Meta: Aumentar market share para >10% em regiões alvo")

# TABELAS DE SAÍDA EXPLÍCITAS
print("\n TABELAS DE SAÍDA PRINCIPAIS:")

print("\n 1️. df_category (primeiras 5 linhas):")
print(df_category[['product_category_name', 'total_revenue', 'revenue_pct', 
                   'segment', 'avg_review_score', 'total_orders']].head().round(2))

print("\n 2️. category_classification (Top 10):")
classification = df_category[['product_category_name', 'segment', 'revenue_pct', 
                              'cumulative_revenue_pct', 'price_volume_segment']].sort_values('revenue_pct', ascending=False)
print(classification.head(10).round(2))

print("\n 3️. priority_categories (Top 5 por priority_score):")
if 'priority_score' in df_nps_analysis.columns:
    priority_table = df_nps_analysis[['product_category_name', 'priority_score', 'revenue_pct', 
                                    'avg_review_score', 'segment']].nlargest(5, 'priority_score')
    print(priority_table.round(2))

print(f"\n" + "="*80)
print("ANÁLISE DE PERFORMANCE POR CATEGORIA CONCLUÍDA")
print("="*80)